In [1]:
pip install transformers torch pandas scikit-learn


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv('chatbot_dataset.csv')

# Display the first few rows
print(df.head())

# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the input texts
def tokenize_function(examples):
    return tokenizer(examples['User Input'], padding='max_length', truncation=True)

# Tokenize the dataset
train_encodings = tokenizer(train_df['User Input'].tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_df['User Input'].tolist(), truncation=True, padding=True, max_length=128)

# Convert labels to numerical format
label_dict = {label: i for i, label in enumerate(df['Response Category'].unique())}
train_labels = [label_dict[label] for label in train_df['Response Category']]
test_labels = [label_dict[label] for label in test_df['Response Category']]

# Convert to torch Dataset
class ChatbotDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ChatbotDataset(train_encodings, train_labels)
test_dataset = ChatbotDataset(test_encodings, test_labels)

# Load the pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_dict))

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()


In [ ]:
def get_response(user_input):
    inputs = tokenizer(user_input, return_tensors='pt', truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=1).item()
    predicted_label = list(label_dict.keys())[list(label_dict.values()).index(predicted_class_id)]
    
    # Find a response corresponding to the predicted label
    response = df[df['Response Category'] == predicted_label]['Bot Response'].values[0]
    return response

# Example usage
while True:
    user_input = input("User: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    response = get_response(user_input)
    print(f"Bot: {response}")
